In [1]:
# # In case the Int Slider does not appear, run:
# !pip install ipywidgets
# !jupyter lab clean
# # restart kernel

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import os
import matplotlib.colors as mcolors
import plotly.graph_objects as go
import plotly.colors as pc

In [3]:
# !pip list --not-required 

# Define results for which dataset to display

In [4]:
# 1. Dataset: Full trips
dataset = "results/Full_trips"

# List all experiment folders

In [5]:
def list_directories(path):
    directories = [f"{path}/{d}" for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]
    return directories

In [6]:
experiments_paths = list_directories(dataset)
experiments_paths

['results/Full_trips/HYBRYD_MODEL_True-LearningR_0.001-epochs_100-patience_100_-TransformUsesCalculFields_False',
 'results/Full_trips/HYBRYD_MODEL_True-LearningR_0.001-epochs_300-patience_300_-TransformUsesCalculFields_False']

# Unify results of all models in each experiment

In [7]:
def get_filepaths_with_extension(file_extension=".csv", directory="."):
    output = []
    
    for file in os.listdir(directory):  
        # Check if the file has the required extension
        if file.endswith(file_extension):
            output.append(f"{directory}/{file}") 
            
    return output

## List all files from first model found

In [8]:
first_model_path = list_directories(experiments_paths[0])[0]
first_model_path += "/test_data_and_model_output"
first_model_path

'results/Full_trips/HYBRYD_MODEL_True-LearningR_0.001-epochs_100-patience_100_-TransformUsesCalculFields_False/Physics/test_data_and_model_output'

In [9]:
first_model_output_filenames = get_filepaths_with_extension(file_extension=".csv", directory=first_model_path)
first_model_output_filenames = [path.split("/")[-1] for path in first_model_output_filenames]    # keep only filename
first_model_output_filenames

['file-0.csv',
 'file-1.csv',
 'file-2.csv',
 'file-3.csv',
 'file-4.csv',
 'file-5.csv',
 'file-6.csv',
 'file-7.csv',
 'file-8.csv',
 'file-9.csv']

## Get and Concatenate results of all models of each experiment

In [10]:
dict_experiments_results = {}

In [11]:
columns_to_keep = ["Time [s]", "SoC [%]", "Physics-and-Transformer", "Physics", "Transformer"]

In [12]:
for experiment_path in experiments_paths:
    # print(experiment_path)
    # get all model directories
    models_directories = list_directories(experiment_path)

    models_output_directories = [f"{model}/test_data_and_model_output" for model in models_directories]    # add directory where models store their output

    # print(models_output_directories)
    
    # get results from each model
    experiment_result = []
    for output_filename in first_model_output_filenames:    # iterate over each filename to make sure the comparison of the different models is correct
        df = None
        for models_output_directory in models_output_directories:    # iterate over the output directories of each model
            csv_filepath = f"{models_output_directory}/{output_filename}"
            df_new = pd.read_csv(csv_filepath, sep=";", encoding="ISO-8859-2")

            # if there is no dataframe stored yet
            if df is None:
                df = df_new.copy()
            else: 
                df = pd.concat([df, df_new], axis=1)    # concatenating along columns 

        # keep only important columns for plots
        df = df[columns_to_keep]    
        # remove duplicated columns
        df = df.loc[:,~df.columns.duplicated()]
        
        # print(df)
        experiment_result.append(df)

    experiment_name = experiment_path.split("/")[-1]
    dict_experiments_results[experiment_name] = experiment_result

In [13]:
dict_experiments_results

{'HYBRYD_MODEL_True-LearningR_0.001-epochs_100-patience_100_-TransformUsesCalculFields_False': [          Time [s]  SoC [%]  Physics-and-Transformer    Physics  Transformer
  0        -0.000017     85.4                86.017990  85.400000     81.23502
  1         0.099959     85.4                86.026306  85.399826     81.20763
  2         0.199934     85.4                86.030690  85.399640     81.13745
  3         0.300032     85.4                86.029760  85.399450     81.09142
  4         0.400008     85.4                86.029110  85.399250     80.97411
  ...            ...      ...                      ...        ...          ...
  13620  1362.000000     71.5                70.651410  68.958250     74.84778
  13621  1362.100000     71.5                70.655970  68.957990     73.85989
  13622  1362.200000     71.5                70.613550  68.957730     73.61963
  13623  1362.300000     71.5                70.560550  68.957466     74.26841
  13624  1362.400000     71.5        

# Create interactive plots

In [14]:
# !pip install plotly

In [15]:
model_output_column_names = ["Physics-and-Transformer", "Transformer", "Physics"]

In [16]:
def plot_soc(file_id, model_outputs_to_display):
    
    # Use Plotly's "Safe" qualitative color palette. Source of color pallete: https://plotly.com/python/discrete-color/
    colorblind_palette = pc.qualitative.Safe

    for experiment_name in list(dict_experiments_results.keys()):
        experiment_results = dict_experiments_results[experiment_name][file_id]
    
        # Create the figure
        fig = go.Figure()
        
        # Plot actual SoC only from first model
        fig.add_trace(go.Scatter(
            x=experiment_results['Time [s]'],
            y=experiment_results['SoC [%]'],
            mode='lines',
            line=dict(width=3, color=colorblind_palette[0]),
            name='Actual SoC [%]'
        ))

        marker_styles = ['circle', 'square', 'diamond', 'cross', 'x', 'triangle-up', 'triangle-down']
        for idx_model, model_output_column_name in enumerate(model_output_column_names):
            # If model_output_column_name is not in selected model_outputs_to_display, skip
            if model_output_column_name not in model_outputs_to_display:
                continue     

            line_opacity = 0.7
            fig.add_trace(go.Scatter(
                x=experiment_results['Time [s]'],
                y=experiment_results[model_output_column_name],
                mode='lines',
                line=dict(dash='dot', width=3, 
                          color=f"rgba{tuple(map(int, colorblind_palette[(idx_model % len(colorblind_palette)) + 1][4:-1].split(',')))[:3] + (line_opacity,)}"),
                name=f"Model: {model_output_column_name}"
            ))    # color with opacity is last answer from AI assistant: https://chatgpt.com/share/67a8fb48-ae30-800b-b9c4-44caa979d630
            
        # Update layout
        fig.update_layout(
            title=f'Actual vs Estimated State of Charge (SoC %) of file {file_id}<br><sup>Experiment: {experiment_name}</sup>',
            xaxis_title='Time [s]',
            yaxis_title='State of Charge [%]',
            legend_title='Legend',
            template='plotly_white'
        )
        
        # Show the plot
        fig.show()

In [17]:
# get total number of files
total_files = len(dict_experiments_results[list(dict_experiments_results)[0]])    # number of dataframes in first experiments


In [18]:
# Create Slider for multiple selection of models
model_select_multiple = widgets.SelectMultiple(
    options=model_output_column_names,
    value=model_output_column_names,
    rows=len(model_output_column_names),
    description='Models:',
    disabled=False
)

# Create a slider for selecting the ID
id_slider = widgets.IntSlider(
    value=0, 
    min=0, 
    max=(total_files -1),    # max is the total number of file in any of the model results 
    step=1, 
    description='Test file ID:',
    continuous_update=False,
    style={'slider_button_width': '20px'}  # Enable built-in buttons
)

# Create a horizontal layout for the id_slider and model_select_multiple
controls = widgets.HBox([model_select_multiple, id_slider])

# Connect the slider to the plot function
interactive_plot = widgets.interactive_output(plot_soc, {'file_id': id_slider, 'model_outputs_to_display': model_select_multiple})

# Display the controls widgets and plot together
display(controls, interactive_plot)

Output()